# import library

In [1]:
# import all library
import librosa, IPython, datetime, time, os, sys, copy, dill, pickle, mir_eval, glob
import numpy as np
import pandas as pd
#from scipy.spatial.distance import euclidean, pdist, squareform
#import IPython.display as ipd
#from datetime import datetime
from time import gmtime, strftime
#from imageio import imread as imread
#from imageio import imsave as imsave
#import librosa.display
import tensorflow as tf
#from pypianoroll import Multitrack, Track
from matplotlib import pyplot as plt
#from midiutil.MidiFile import MIDIFile
#import ray
from ops import *
#import pydub
#from ADTLib import ADT
import matplotlib.pyplot as plt
%matplotlib inline

print ("[info] Current Time:     " + datetime.datetime.now().strftime('%Y/%m/%d  %H:%M:%S'))
print ("[info] Python Version:   " + sys.version.split('\n')[0].split(' ')[0])
print ("[info] Working Dir:      " + os.getcwd()+'/')
print('[info] Tensorflow Version: {}'.format(tf.__version__))
# enable gpu usage constraint here
fixed_gpu_usage = 1

# if gpu usage is constraint, limit certain gpu for use
if (fixed_gpu_usage == 1):
    # set available GPU
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"        # list GPU sequence by PCI bus GPU ID
    #os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"         # 0 for GPU 0(1080Ti), 1 for GPU 1(1080Ti), 2 for GPU 2(2080Ti)
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"              # 0 for GPU 0(1080Ti), 1 for GPU 1(1080Ti), 2 for GPU 2(2080Ti)

    # check available GPU
    from tensorflow.python.client import device_lib
    for x in range(1, len(device_lib.list_local_devices())):
        print ("[info] GPU " + device_lib.list_local_devices()[x].physical_device_desc)

[info] Current Time:     2019/03/21  04:51:03
[info] Python Version:   3.6.5
[info] Working Dir:      /host/home/python/musegan_npz/
[info] Tensorflow Version: 1.8.0
[info] GPU device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1


# define gather function

In [2]:
def tf_broadcast(tensor, shape):
    return (tensor + tf.zeros(shape, dtype=tensor.dtype))

def get_idx_base(batch_size_v,
                 matx_size):
    
    mtx_len_np = np.array([matx_size]).astype(np.int32)
    
    np_init_idx = np.zeros([(matx_size**2)*batch_size_v, 1]).astype(np.int32)
    np_init_idx[:,0] = np.arange((matx_size**2)*batch_size_v)//((matx_size**2)*batch_size_v//batch_size_v)
    
    with tf.variable_scope("index_op_{}".format(str(time.time())[5:10]), reuse=tf.AUTO_REUSE):
        
        index_bat_2d_out = tf.get_variable("tfv_batch_pos", 
                                initializer=np_init_idx,
                                dtype="int32")
        
        
        np_init_idx_base = ((np.arange(0,mtx_len_np*mtx_len_np)//mtx_len_np)*mtx_len_np).astype(np.int32)
        index_base_v = tf.get_variable("tfv_index_base_{}".format(str(time.time())[5:10]),
                                       initializer=np_init_idx_base,
                                       dtype="int32")
        index_base_v_2d_out = tf.tile(tf.reshape(index_base_v, [1,-1]), [batch_size_v,1])
        
        
        np_init_idx_inc = np.tile(np.arange(0,mtx_len_np),mtx_len_np).astype(np.int32)
        index_inc_v = tf.get_variable("tfv_index_inc_{}".format(str(time.time())[5:10]),
                                       initializer=np_init_idx_inc,
                                       dtype="int32")
        index_inc_v_2d_out = tf.tile(tf.reshape(index_inc_v, [1,-1]), [batch_size_v,1])
        
        
        np_init_idx_sub = np.tile(np.arange(0,mtx_len_np).reshape(-1,1),[1,mtx_len_np[0]]).astype(np.int32).reshape([-1])
        index_sub_v = tf.get_variable("tfv_index_sub_{}".format(str(time.time())[5:10]),
                                       initializer=np_init_idx_sub,
                                       dtype="int32")
        index_sub_v_2d_out = tf.tile(tf.reshape(index_sub_v, [1,-1]), [batch_size_v,1])        
        
    return index_bat_2d_out, index_base_v_2d_out, index_inc_v_2d_out, index_sub_v_2d_out


def gather_data_song_len_ary(data_in3d,
                             song_len_2d_in,
                             param_batch_size,
                             param_mtx_len_int):
    
    index_bat_2d,                                      \
    index_base_v_2d,                                   \
    index_inc_v_2d,                                    \
    index_sub_v_2d = get_idx_base(param_batch_size,    \
                                  param_mtx_len_int)

    song_len_tfv_2d = tf_broadcast(tf.reshape(song_len_2d_in,[-1,1]), tf.shape(index_sub_v_2d))

    index_sub_2d = tf.div(index_sub_v_2d, song_len_tfv_2d) * song_len_tfv_2d * mtx_len_int

    index_pos_2d = index_base_v_2d + tf.mod(index_inc_v_2d, song_len_tfv_2d) - index_sub_2d
    index_pos_2d_flat = tf.reshape(index_pos_2d, [-1,1])

    index_ary_2d = tf.concat([index_bat_2d[:,0:1],     \
                              index_pos_2d_flat],      \
                             axis=1)


    # reshape data first
    data_in3d_flat = tf.reshape(data_in3d, [data_in3d.shape[0], -1])
    
    # gather all values
    data_in3d_flat_gnd = tf.gather_nd(data_in3d_flat,
                                      index_ary_2d)

    # reshape to original matrix
    data_in3d_gnd_rb = tf.reshape(data_in3d_flat_gnd, 
                             [data_in3d.shape[0], 
                              data_in3d.shape[1], 
                              data_in3d.shape[2]])
    
    return data_in3d_gnd_rb

print ('[info] gather function defined.')

[info] gather function defined.


# define trangle extraction function

In [10]:
# accepts only 3-D tensor
def tf_get_tri_rfv_mtx(tgrrm_tensor_in):
    
    a_2d_triu1 = tf.matrix_band_part(tgrrm_tensor_in, 0, -1)
    a_2d_triu2 = tf.transpose(a_2d_triu1, [0,2,1])
    a_2d_diag = tf.matrix_band_part(tgrrm_tensor_in, 0, 0)
    a_2d_mix = a_2d_triu1 + a_2d_triu2 - a_2d_diag
    
    return(a_2d_mix)

print ('[info] triangle function defined.')

[info] triangle function defined.


# prepare input data (a_3d1)

In [11]:
mtx_len_int = 15

batch_data1 = tf.reshape(tf.range(mtx_len_int**2), [mtx_len_int, mtx_len_int])
batch_data2 = tf.reshape(tf.range(mtx_len_int**2), [mtx_len_int, mtx_len_int]) + (mtx_len_int**2)*1
batch_data3 = tf.reshape(tf.range(mtx_len_int**2), [mtx_len_int, mtx_len_int]) + (mtx_len_int**2)*2

batch_data1 = tf.reshape(batch_data1, [1, batch_data1.shape[0], batch_data1.shape[1]])
batch_data2 = tf.reshape(batch_data2, [1, batch_data2.shape[0], batch_data2.shape[1]])
batch_data3 = tf.reshape(batch_data3, [1, batch_data3.shape[0], batch_data3.shape[1]])

a_3d1 = tf.concat([batch_data1, 
                   batch_data2,
                   batch_data3], axis=0)

print (a_3d1.shape)

(3, 15, 15)


# calculate gathered data

In [12]:
# batch size = int(3) in this test case
batch_size = a_3d1.get_shape().as_list()[0]


# get 2d song_len variable
song_len_bat = [6,3,4]
np_init_song_len = np.array(song_len_bat).astype(np.int32)
song_len_tfv = tf.get_variable("tfv_song_len_{}".format(str(time.time())[5:10]),
                               initializer=np_init_song_len,
                               dtype="int32")


# get gather data from song_len_2d
a_3d1_gn_rb = gather_data_song_len_ary(a_3d1,
                                       song_len_tfv,
                                       batch_size,
                                       mtx_len_int)



a_3d1_tri = tf_get_tri_rfv_mtx(a_3d1)


# start session and check result
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


# print original input data
print('a_3d1:')
print(a_3d1.get_shape())
print(a_3d1.eval())
print ('\n')


# print original input data
print('a_3d1_tri:')
print(a_3d1_tri.get_shape())
print(a_3d1_tri.eval())
print ('\n')



# print original input data
print('song_len_tfv:')
print(song_len_tfv.get_shape())
print(song_len_tfv.eval())
print ('\n')


# print gathered data
print('a_3d3_gather_reshape_back:')
print(a_3d1_gn_rb.get_shape())
print(a_3d1_gn_rb.eval())
print ('\n')


# close session to avoid OOM
sess.close()

a_3d1:
(3, 15, 15)
[[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14]
  [ 15  16  17  18  19  20  21  22  23  24  25  26  27  28  29]
  [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44]
  [ 45  46  47  48  49  50  51  52  53  54  55  56  57  58  59]
  [ 60  61  62  63  64  65  66  67  68  69  70  71  72  73  74]
  [ 75  76  77  78  79  80  81  82  83  84  85  86  87  88  89]
  [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104]
  [105 106 107 108 109 110 111 112 113 114 115 116 117 118 119]
  [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134]
  [135 136 137 138 139 140 141 142 143 144 145 146 147 148 149]
  [150 151 152 153 154 155 156 157 158 159 160 161 162 163 164]
  [165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]
  [180 181 182 183 184 185 186 187 188 189 190 191 192 193 194]
  [195 196 197 198 199 200 201 202 203 204 205 206 207 208 209]
  [210 211 212 213 214 215 216 217 218 219 220 221 222 223 224]]

 [[225 226 227 228 

# Advanced tensorflow slicing - single

In [3]:
batch_size_v = a_3d1.get_shape().as_list()[0]

song_len_int = 6
np_init_song_len = np.array([song_len_int]).astype(np.int32)
song_len_tfv = tf.get_variable("tfv_song_len_{}".format(str(time.time())[5:10]),
                               initializer=np_init_song_len,
                               dtype="int32")


def get_2d_bat_idx(batch_size=batch_size_v,
                   matx_size=data_mtx_len_int):
    
    np_init_idx = np.zeros([(matx_size**2)*batch_size, 1]).astype(np.int32)
    np_init_idx[:,0] = np.arange((matx_size**2)*batch_size)//((matx_size**2)*batch_size // batch_size)
    with tf.variable_scope("indexop_{}".format(str(time.time())[5:10]), reuse=tf.AUTO_REUSE):
        v_out = tf.get_variable("tfv_batch_pos", 
                                initializer=np_init_idx,
                                dtype="int32")
    return v_out



np_init_idx_base = ((np.arange(0,data_mtx_len*data_mtx_len)//data_mtx_len)*data_mtx_len).astype(np.int32)
index_base_v = tf.get_variable("tfv_index_base_{}".format(str(time.time())[5:10]),
                               initializer=np_init_idx_base,
                               dtype="int32")

np_init_idx_inc = np.tile(np.arange(0,data_mtx_len),data_mtx_len).astype(np.int32)
index_inc_v = tf.get_variable("tfv_index_inc_{}".format(str(time.time())[5:10]),
                               initializer=np_init_idx_inc,
                               dtype="int32")

np_init_idx_sub = np.tile(np.arange(0,data_mtx_len).reshape(-1,1),[1,data_mtx_len[0]]).astype(np.int32).reshape([-1])
index_sub_v = tf.get_variable("tfv_index_sub_{}".format(str(time.time())[5:10]),
                               initializer=np_init_idx_sub,
                               dtype="int32")



index_sub_1d = tf.div(index_sub_v, song_len_tfv) * song_len_tfv * data_mtx_len

index_pos_1d = index_base_v + tf.mod(index_inc_v, song_len_tfv) - index_sub_1d

index_pos_2d = tf.reshape(tf.tile(index_pos_1d, [batch_size_v]), [-1,1])

index_bat_2d = get_2d_bat_idx()

index_ary_2d = tf.concat([index_bat_2d[:,0:1], 
                          index_pos_2d], 
                         axis=1)


# get all values
a_3d1_flat_gn = tf.gather_nd(a_3d1_flat,
                             index_ary_2d)

# reshape to original matrix
a_3d1_gn_rb = tf.reshape(a_3d1_flat_gn, 
                         [a_3d1.shape[0], 
                          a_3d1.shape[1], 
                          a_3d1.shape[2]])



# start session and check result
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


print('a_3d1:')
print(a_3d1.get_shape())
print(a_3d1.eval())
print ('\n')

#print(index_sub_v_mix_flat.shape)
#print(index_sub_v_mix_flat.eval())
#print ('\n')

#print(index_base_inc.shape)
#print(index_base_inc.eval())
#print ('\n')

#print(index_base_inc_full.shape)
#print(index_base_inc_full.eval())
#print ('\n')

#print(song_len_tfv.shape)
#print(song_len_tfv.eval())
#print ('\n')

print('a_3d3_gather_reshape_back:')
print(a_3d1_gn_rb.eval())
print ('\n')

sess.close()

a_3d1:
(3, 15, 15)
[[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14]
  [ 15  16  17  18  19  20  21  22  23  24  25  26  27  28  29]
  [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44]
  [ 45  46  47  48  49  50  51  52  53  54  55  56  57  58  59]
  [ 60  61  62  63  64  65  66  67  68  69  70  71  72  73  74]
  [ 75  76  77  78  79  80  81  82  83  84  85  86  87  88  89]
  [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104]
  [105 106 107 108 109 110 111 112 113 114 115 116 117 118 119]
  [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134]
  [135 136 137 138 139 140 141 142 143 144 145 146 147 148 149]
  [150 151 152 153 154 155 156 157 158 159 160 161 162 163 164]
  [165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]
  [180 181 182 183 184 185 186 187 188 189 190 191 192 193 194]
  [195 196 197 198 199 200 201 202 203 204 205 206 207 208 209]
  [210 211 212 213 214 215 216 217 218 219 220 221 222 223 224]]

 [[225 226 227 228 

# check data format

In [23]:
print(song_len_tfv.shape)
print(index_sub_1d.shape)
print(index_pos_1d.shape)
print(index_pos_2d.shape)
print(index_base_v.shape)

(3,)
(225, 3)
(225,)
(675, 1)
(225,)


In [11]:
song_len_bat = [6,3,4]
np_init_song_len = np.array(song_len_bat).astype(np.int32)

In [12]:
np_init_song_len

array([6, 3, 4], dtype=int32)

In [19]:
print(song_len_tfv.shape)
print(index_sub_v_2d.shape)

(3,)
(3, 225)


# define reshape function

# Advanced tensorflow slicing - batch

In [3]:
## batch size = int(3) in this test case
#batch_size = a_3d1.get_shape().as_list()[0]


#song_len_bat = [6,3,4]
#np_init_song_len = np.array(song_len_bat).astype(np.int32)
#song_len_2d_tfv = tf.get_variable("tfv_song_len_{}".format(str(time.time())[5:10]),
#                                  initializer=np_init_song_len,
#                                  dtype="int32")


def tf_broadcast(tensor, shape):
    return (tensor + tf.zeros(shape, dtype=tensor.dtype))

def get_idx_base(batch_size_v,
                 matx_size):
    
    mtx_len_np = np.array([matx_size]).astype(np.int32)
    
    np_init_idx = np.zeros([(matx_size**2)*batch_size_v, 1]).astype(np.int32)
    np_init_idx[:,0] = np.arange((matx_size**2)*batch_size_v)//((matx_size**2)*batch_size_v//batch_size_v)
    
    with tf.variable_scope("index_op_{}".format(str(time.time())[5:10]), reuse=tf.AUTO_REUSE):
        
        index_bat_2d_out = tf.get_variable("tfv_batch_pos", 
                                initializer=np_init_idx,
                                dtype="int32")
        
        
        np_init_idx_base = ((np.arange(0,mtx_len_np*mtx_len_np)//mtx_len_np)*mtx_len_np).astype(np.int32)
        index_base_v = tf.get_variable("tfv_index_base_{}".format(str(time.time())[5:10]),
                                       initializer=np_init_idx_base,
                                       dtype="int32")
        index_base_v_2d_out = tf.tile(tf.reshape(index_base_v, [1,-1]), [batch_size_v,1])
        
        
        np_init_idx_inc = np.tile(np.arange(0,mtx_len_np),mtx_len_np).astype(np.int32)
        index_inc_v = tf.get_variable("tfv_index_inc_{}".format(str(time.time())[5:10]),
                                       initializer=np_init_idx_inc,
                                       dtype="int32")
        index_inc_v_2d_out = tf.tile(tf.reshape(index_inc_v, [1,-1]), [batch_size_v,1])
        
        
        np_init_idx_sub = np.tile(np.arange(0,mtx_len_np).reshape(-1,1),[1,mtx_len_np[0]]).astype(np.int32).reshape([-1])
        index_sub_v = tf.get_variable("tfv_index_sub_{}".format(str(time.time())[5:10]),
                                       initializer=np_init_idx_sub,
                                       dtype="int32")
        index_sub_v_2d_out = tf.tile(tf.reshape(index_sub_v, [1,-1]), [batch_size_v,1])        
        
    return index_bat_2d_out, index_base_v_2d_out, index_inc_v_2d_out, index_sub_v_2d_out


def gather_data_song_len_ary(data_in3d,
                             song_len_2d_in,
                             param_batch_size,
                             param_mtx_len_int
                            ):
    
    index_bat_2d,                                      \
    index_base_v_2d,                                   \
    index_inc_v_2d,                                    \
    index_sub_v_2d = get_idx_base(param_batch_size,    \
                                  param_mtx_len_int)

    song_len_tfv_2d = tf_broadcast(tf.reshape(song_len_2d_in,[-1,1]), tf.shape(index_sub_v_2d))

    index_sub_2d = tf.div(index_sub_v_2d, song_len_tfv_2d) * song_len_tfv_2d * mtx_len_int

    index_pos_2d = index_base_v_2d + tf.mod(index_inc_v_2d, song_len_tfv_2d) - index_sub_2d
    index_pos_2d_flat = tf.reshape(index_pos_2d, [-1,1])

    index_ary_2d = tf.concat([index_bat_2d[:,0:1],     \
                              index_pos_2d_flat],      \
                             axis=1)


    # reshape data first
    data_in3d_flat = tf.reshape(data_in3d, [data_in3d.shape[0], -1])
    
    # gather all values
    data_in3d_flat_gnd = tf.gather_nd(data_in3d_flat,
                                      index_ary_2d)

    # reshape to original matrix
    data_in3d_gnd_rb = tf.reshape(data_in3d_flat_gnd, 
                             [data_in3d.shape[0], 
                              data_in3d.shape[1], 
                              data_in3d.shape[2]])
    
    return data_in3d_gnd_rb



# batch size = int(3) in this test case
batch_size = a_3d1.get_shape().as_list()[0]


song_len_bat = [6,3,4]
np_init_song_len = np.array(song_len_bat).astype(np.int32)
song_len_2d_tfv = tf.get_variable("tfv_song_len_{}".format(str(time.time())[5:10]),
                                  initializer=np_init_song_len,
                                  dtype="int32")

a_3d1_gn_rb = gather_data_song_len_ary(a_3d1,
                                       song_len_2d_tfv,
                                       batch_size,
                                       mtx_len_int)


# start session and check result
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


#print('a_3d1:')
#print(a_3d1.get_shape())
#print(a_3d1.eval())
#print ('\n')

#print(index_sub_v_mix_flat.shape)
#print(index_sub_v_mix_flat.eval())
#print ('\n')

#print(index_base_inc.shape)
#print(index_base_inc.eval())
#print ('\n')

#print(index_base_inc_full.shape)
#print(index_base_inc_full.eval())
#print ('\n')

#print(song_len_tfv.shape)
#print(song_len_tfv.eval())
#print ('\n')

print('a_3d3_gather_reshape_back:')
print(a_3d1_gn_rb[:,:,:].eval())
print ('\n')

sess.close()

a_3d3_gather_reshape_back:
[[[  0   1   2   3   4   5   0   1   2   3   4   5   0   1   2]
  [ 15  16  17  18  19  20  15  16  17  18  19  20  15  16  17]
  [ 30  31  32  33  34  35  30  31  32  33  34  35  30  31  32]
  [ 45  46  47  48  49  50  45  46  47  48  49  50  45  46  47]
  [ 60  61  62  63  64  65  60  61  62  63  64  65  60  61  62]
  [ 75  76  77  78  79  80  75  76  77  78  79  80  75  76  77]
  [  0   1   2   3   4   5   0   1   2   3   4   5   0   1   2]
  [ 15  16  17  18  19  20  15  16  17  18  19  20  15  16  17]
  [ 30  31  32  33  34  35  30  31  32  33  34  35  30  31  32]
  [ 45  46  47  48  49  50  45  46  47  48  49  50  45  46  47]
  [ 60  61  62  63  64  65  60  61  62  63  64  65  60  61  62]
  [ 75  76  77  78  79  80  75  76  77  78  79  80  75  76  77]
  [  0   1   2   3   4   5   0   1   2   3   4   5   0   1   2]
  [ 15  16  17  18  19  20  15  16  17  18  19  20  15  16  17]
  [ 30  31  32  33  34  35  30  31  32  33  34  35  30  31  32]]

 [[225 226 

In [8]:
batch_size

3

In [45]:
tf.reshape(tf.range(data_mtx_len[0]*data_mtx_len[0]), [data_mtx_len[0], data_mtx_len[0]]) + 25

<tf.Tensor 'add_1:0' shape=(5, 5) dtype=int32>

In [34]:
data_mtx_len = 5
(np.arange(0,data_mtx_len*data_mtx_len)//data_mtx_len)*data_mtx_len

array([ 0,  0,  0,  0,  0,  5,  5,  5,  5,  5, 10, 10, 10, 10, 10, 15, 15,
       15, 15, 15, 20, 20, 20, 20, 20])

In [26]:
((np.arange(0,4*4)//4)*4).reshape(4,4)

array([[ 0,  0,  0,  0],
       [ 4,  4,  4,  4],
       [ 8,  8,  8,  8],
       [12, 12, 12, 12]])

In [37]:
np.tile(np.arange(0,6),6)

array([0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4,
       5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5])

In [38]:
np.tile(np.arange(0,6),(6,1)) % 3

array([[0, 1, 2, 0, 1, 2],
       [0, 1, 2, 0, 1, 2],
       [0, 1, 2, 0, 1, 2],
       [0, 1, 2, 0, 1, 2],
       [0, 1, 2, 0, 1, 2],
       [0, 1, 2, 0, 1, 2]])

In [31]:
(((np.arange(0,4*4)//4)*4) + (np.tile(np.arange(0,4),4)) % 2).reshape(4,4)

array([[ 0,  1,  0,  1],
       [ 4,  5,  4,  5],
       [ 8,  9,  8,  9],
       [12, 13, 12, 13]])

In [80]:
(np.tile(np.arange(8).reshape(-1,1),[1,8])//5)*8*5

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [40, 40, 40, 40, 40, 40, 40, 40],
       [40, 40, 40, 40, 40, 40, 40, 40],
       [40, 40, 40, 40, 40, 40, 40, 40]])

# Check padding function

In [ ]:
def tf_periodic_padding_flexible(tensor, axis,padding=1):
    """
        add periodic padding to a tensor for specified axis
        tensor: input tensor
        axis: on or multiple axis to pad along, int or tuple
        padding: number of cells to pad, int or tuple

        return: padded tensor
    """
    if isinstance(axis,int):
        axis = (axis,)
    if isinstance(padding,int):
        padding = (padding,)

    ndim = len(tensor.shape)
    for ax,p in zip(axis,padding):
        # create a slice object that selects everything from all axes,
        # except only 0:p for the specified for right, and -p: for left

        ind_right = [slice(-p,None) if i == ax else slice(None) for i in range(ndim)]
        ind_left = [slice(0, p) if i == ax else slice(None) for i in range(ndim)]
        right = tensor[ind_right]
        left = tensor[ind_left]
        middle = tensor
        #tensor = tf.concat([right,middle,left], axis=ax)
        tensor = tf.concat([middle,left], axis=ax)

    return tensor


def tf_get_tri_rfv_mtx(tgrrm_tensor_in):
    
    a_2d_triu1 = tf.matrix_band_part(tgrrm_tensor_in, 0, -1)
    a_2d_triu2 = tf.transpose(a_2d_triu1)
    a_2d_diag = tf.matrix_band_part(a_2d, 0, 0)
    a_2d_mix = a_2d_triu1 + a_2d_triu2 - a_2d_diag
    
    return(a_2d_mix)


def get_target_shape_tesnsor(gtst_tensor_in, target_size_in):

    #gtst_tensor_in_size = a_2d_mix.get_shape()[0]
    gtst_tensor_in_size = tf.shape(a_2d_mix)[0]
    ax_mulv = (target_size_in+gtst_tensor_in_size-1)//gtst_tensor_in_size
    ax_mulv_2d = tf.reshape(tf.stack([ax_mulv, ax_mulv]), [-1])

    a_2d_tile_out = tf.tile(a_2d_mix, 
                            ax_mulv_2d,
                            name='tftile_func')
    
    return(a_2d_tile_out[:target_size_in,:target_size_in])
    #return(a_2d_tile_out[:,:])
    
    

# start calculation here
a_2d = tf.constant([[ 1, 2, 3, 4],
                    [ 5, 6, 7, 8],
                    [ 9,10,11,12],
                    [13,14,15,16]])

a_3d = tf.constant([[[ 1, 2, 3, 4],
                     [ 5, 6, 7, 8],
                     [ 9,10,11,12],
                     [13,14,15,16]],
                  
                    [[17,18,19,20],
                     [21,22,23,24],
                     [25,26,27,28],
                     [29,30,31,32]]])                  
                  
                  
            


nRows = 2
nCols = 2
m1 = tf.reshape(tf.tile(tf.range(nCols), [nRows]),
                                        shape=[nRows, nCols])

m2 = tf.transpose(tf.reshape(tf.tile(tf.range(nRows), [nCols]),
                                        shape=[nCols, nRows]))

m12_indices = tf.stack([m2, m1], axis=-1)

#slice_index = tf.tile(tf.range(2), [2,2])
#slice_index = 
#a_2d_gather = slice_index
#a_2d_gather_idx = m12_indices

a_2d_gather_idx = [[[0,0], [0,1]], [[0,0], [0,1]]]

a_2d_gather = tf.gather_nd(a_2d, a_2d_gather_idx)

a_2d_mix = tf_get_tri_rfv_mtx(a_2d)

a_2d_mix_slice = tf.slice(a_2d_mix, 
                          [0, 0],
                          [-1, 3])

a_2d_mix_tile = get_target_shape_tesnsor(a_2d_mix, 9)

#a_3d = tf.constant([
#                    [[1,2,3],[4,5,6],[7,8,9]],
#                    [[11,12,13],[14,15,16],[17,18,19]],
#                    ])

sess = tf.InteractiveSession()


print('a_2d:')
print(a_2d.eval())
print ('\n')

print('m1:')
print(m1.eval())
print ('\n')

print('m2:')
print(m2.eval())
print ('\n')

print('m12_indices:')
print(m12_indices.eval())
print ('\n')

print('a_2d_gather:')
print(a_2d_gather.eval())
print ('\n')

#print('a_2d_triu2:')
#print(a_2d_triu2.eval())
#print ('\n')

#print('a_2d_diag:')
#print(a_2d_diag.eval())
#print ('\n')

print('a_2d_mix:')
print(a_2d_mix.eval())
print ('\n')

print('a_2d_mix_slice:')
print(a_2d_mix_slice.eval())
print ('\n')

print('a_2d_mix_tile_cut:')
print(a_2d_mix_tile.eval())
print ('\n')


#a_2d_mix_padded1 = tf_periodic_padding_flexible(a_2d_mix, axis=(0,1), padding=(2,2))
#print('padded a1:')
#print(a_2d_mix_padded1.eval())

#a_2d_mix_padded2 = tf_periodic_padding_flexible(a_2d_mix, axis=(0,1), padding=(6,6))
#print('padded a1:')
#print(a_2d_mix_padded2.eval())


# close session afer use to avoid OOM error
sess.close()

In [30]:
a_2d = tf.constant([[ 1, 2, 3, 4],
                    [ 5, 6, 7, 8],
                    [ 9,10,11,12],
                    [13,14,15,16]])

#a_2d_idx = tf.constant([[ 0, 0, 0, 0],
#                        [ 0, 1, 1, 0],
#                        [ 0, 1, 1, 0],
#                        [ 0, 0, 0, 0]])

a_2d_idx = tf.constant([[ 0],
                        [ 1],
                        [ 1],
                        [ 0]])


a_3d1 = tf.constant([[[ 1, 2, 3, 4],
                      [ 5, 6, 7, 8],
                      [ 9,10,11,12],
                      [13,14,15,16]],
                   
                     [[17,18,19,20],
                      [21,22,23,24],
                      [25,26,27,28],
                      [29,30,31,32]]])   

a_4d1 = tf.expand_dims(a_3d1, -1)

#a_3d2 = tf.constant([[[ 1, 1, 0, 0],
#                      [ 1, 1, 0, 0],
#                      [ 0, 0, 0, 0],
#                      [ 0, 0, 0, 0]],
#                  
#                     [[ 1, 1, 1, 0],
#                      [ 1, 1, 1, 0],
#                      [ 1, 1, 1, 0],
#                      [ 0, 0, 0, 0]]])   

a_3d1_flat = tf.reshape(a_3d1, [a_3d1.shape[0], -1])
#a_3d2_flat = tf.reshape(a_3d2, [a_3d2.shape[0], -1])

indices = tf.constant([[0,0],[0,1],[0,4],[0,5],
                       [1,0],[1,1],[1,4],[1,5]])

a_3d1_flat_bg = tf.gather_nd(a_3d1_flat,
                             indices,
                             name=None)

#a_3d2_flat_bg = tf.gather_nd(a_3d2_flat,
#                             indices,
#                             name=None)


a_3d1_gather_reshape_back = tf.reshape(a_3d1_flat_bg, [a_3d1.shape[0], -1])
#a_3d2_gather_reshape_back = tf.reshape(a_3d2_flat_bg, [a_3d2.shape[0], -1])


#a_3d_idx = tf.constant([[[ 0, 0, 0, 0],
#                         [ 0, 1, 1, 0],
#                         [ 0, 0, 0, 0],
#                         [ 0, 0, 0, 0]],
#                  
#                        [[ 0, 0, 0, 0],
#                         [ 0, 1, 1, 0],
#                         [ 0, 0, 0, 0],
#                         [ 0, 0, 0, 0]]])   

zeross = tf.ones([1], dtype='uint8') + 255

index_ary = tf.cast(tf.Variable(tf.range(16, dtype="int32"),dtype="int32"), tf.uint8)

index_ary_2d = tf.expand_dims(index_ary, -1)
#index_ary_2d_exp = tf.tile(index_ary_2d, 
#                          [1,2],
#                          name='tftile_func')


#np_init_idx = np.zeros([4,4]).astype(np.uint8)
#index_ary_2d_exp = tf.get_variable("ary_index_1",
#                                   initializer=np_init_idx,
#                                   dtype="uint8")



def ary_index():
    batch_size = 2
    matrix_size = 4
    np_init_idx = np.zeros([(matrix_size**2)*2, 2]).astype(np.int32)
    np_init_idx[:,0] = np.arange((matrix_size**2)*2)//((matrix_size**2)*2 // batch_size)
    np_init_idx[14,1] = 4
    np_init_idx[24,1] = 5
    with tf.variable_scope("ary_index_{}".format(str(time.time())[5:10]), reuse=tf.AUTO_REUSE):
        v = tf.get_variable("tf_var_np_init", 
                            initializer=np_init_idx,
                            dtype="int32")
    return v

index_ary_2d_exp = ary_index()


a_3d3_flat_bg = tf.gather_nd(a_3d1_flat,
                             index_ary_2d_exp,
                             name=None)

a_3d3_gather_reshape_back = tf.reshape(a_3d3_flat_bg, [a_3d1.shape[0], a_3d1.shape[1], a_3d1.shape[2]])

#index_ary_2d_exp.assign(0)

#index_ary_2d_exp = index_ary_2d_exp * 0

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


print('a_3d1:')
print(a_3d1.get_shape())
print(a_3d1.eval())
print ('\n')

#print(index_ary.eval())
#print ('\n')

print(index_ary_2d_exp.shape)
print(index_ary_2d_exp.eval())
#print(index_ary_2d_exp[-5:,:].eval())
print ('\n')

#print('a_4d1:')
#print(a_4d1.get_shape())
#print(a_4d1.eval())
#print ('\n')


#print('a_3d2:')
#print(a_3d2.eval())
#print ('\n')

#print('a_3d1_flat:')
#print(a_3d1_flat.eval())
#print ('\n')

#print('a_3d2_flat:')
#print(a_3d2_flat.eval())
#print ('\n')

#print('a_3d1_flat_bg:')
#print(a_3d1_flat_bg.eval())
#print ('\n')

#print('a_3d3_flat_bg:')
#print(a_3d3_flat_bg.eval())
#print ('\n')

#print('a_3d2_flat_bg:')
#print(a_3d2_flat_bg.eval())
#print ('\n')

#print('a_3d1_gather_reshape_back:')
#print(a_3d1_gather_reshape_back.eval())
#print ('\n')

print('a_3d3_gather_reshape_back:')
print(a_3d3_gather_reshape_back.eval())
print ('\n')

#print('a_3d2_gather_reshape_back:')
#print(a_3d2_gather_reshape_back.eval())
#print ('\n')


# close session afer use to avoid OOM error
sess.close()

a_3d1:
(2, 4, 4)
[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]
  [13 14 15 16]]

 [[17 18 19 20]
  [21 22 23 24]
  [25 26 27 28]
  [29 30 31 32]]]


(32, 2)
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 4]
 [0 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 5]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


a_3d3_gather_reshape_back:
[[[ 1  1  1  1]
  [ 1  1  1  1]
  [ 1  1  1  1]
  [ 1  1  5  1]]

 [[17 17 17 17]
  [17 17 17 17]
  [22 17 17 17]
  [17 17 17 17]]]




/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [31]:
a_3d1 = tf.constant([[[ 1, 2, 3, 4],
                      [ 5, 6, 7, 8],
                      [ 9,10,11,12],
                      [13,14,15,16]],
                   
                     [[17,18,19,20],
                      [21,22,23,24],
                      [25,26,27,28],
                      [29,30,31,32]]])   


def ary_index():
    batch_size = 2
    matrix_size = 4
    np_init_idx = np.zeros([(matrix_size**2)*2, 2]).astype(np.int32)
    np_init_idx[:,0] = np.arange((matrix_size**2)*2)//((matrix_size**2)*2 // batch_size)
    np_init_idx[14,1] = 4
    np_init_idx[24,1] = 5
    with tf.variable_scope("ary_index_{}".format(str(time.time())[5:10]), reuse=tf.AUTO_REUSE):
        v = tf.get_variable("tf_var_np_init", 
                            initializer=np_init_idx,
                            dtype="int32")
    return v

index_ary_2d = ary_index()


a_3d3_flat_bg = tf.gather_nd(a_3d1_flat,
                             index_ary_2d_exp,
                             name=None)

a_3d3_gather_reshape_back = tf.reshape(a_3d3_flat_bg, [a_3d1.shape[0], a_3d1.shape[1], a_3d1.shape[2]])


sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


print('a_3d1:')
print(a_3d1.get_shape())
print(a_3d1.eval())
print ('\n')


print(index_ary_2d_exp.shape)
print(index_ary_2d_exp.eval())
print ('\n')


print('a_3d3_gather_reshape_back:')
print(a_3d3_gather_reshape_back.eval())
print ('\n')


sess.close()

a_3d1:
(2, 4, 4)
[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]
  [13 14 15 16]]

 [[17 18 19 20]
  [21 22 23 24]
  [25 26 27 28]
  [29 30 31 32]]]


(32, 2)
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 4]
 [0 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 5]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


a_3d3_gather_reshape_back:
[[[ 1  1  1  1]
  [ 1  1  1  1]
  [ 1  1  1  1]
  [ 1  1  5  1]]

 [[17 17 17 17]
  [17 17 17 17]
  [22 17 17 17]
  [17 17 17 17]]]




/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
